In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import time

In [2]:
model = load_model('drowsiness_model.h5')  # adjust filename if different

In [3]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [11]:
cap = cv2.VideoCapture(0)
closed_eyes_counter = 0
threshold = 40  # adjust depending on frame rate

In [13]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=6)

    for (ex, ey, ew, eh) in eyes:
        eye = gray_frame[ey:ey+eh, ex:ex+ew]
        
        # Fix 2: Resize to 48x48 to match model's input shape
        eye = cv2.resize(eye, (48, 48))  # Changed from 24x24
        
        # Fix 3: Ensure correct channel dimension
        eye = eye.astype("float32") / 255.0
        eye = np.expand_dims(eye, axis=-1)  # Add channel dimension (48,48,1)
        eye = np.expand_dims(eye, axis=0)   # Add batch dimension (1,48,48,1)

        prediction = model.predict(eye, verbose=0)[0]
        open_prob = prediction[1]
        label = "Open" if open_prob > 0.4 else "Closed"

        # Drowsiness logic
        if label == "Closed":
            closed_eyes_counter += 1
        else:
            closed_eyes_counter = 0

        if closed_eyes_counter >= threshold:
            cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        # Draw annotations
        color = (0, 255, 0) if label == "Open" else (0, 0, 255)
        cv2.rectangle(frame, (ex, ey), (ex+ew, ey+eh), color, 2)
        cv2.putText(frame, label, (ex, ey-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()